In [1]:
import os
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import center_of_mass, shift
import json
from sklearn.preprocessing import MinMaxScaler

def resample_image(itk_image, out_spacing, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    original_direction = itk_image.GetDirection()
    original_origin = itk_image.GetOrigin()

    out_size = [
        int(round(osz * ospc / nspc))
        for osz, ospc, nspc in zip(itk_image.GetSize(), original_spacing, out_spacing)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(out_spacing)
    resampler.SetSize(out_size)
    resampler.SetOutputDirection(original_direction)
    resampler.SetOutputOrigin(original_origin)
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(0)

    if is_label:
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        # resampler.SetInterpolator(sitk.sitkLabelGaussian)
        # resampler.SetSigma(3.0)   # 控制边界模糊的程度，数值越大越模糊（默认1.0）
    else:
        resampler.SetInterpolator(sitk.sitkBSpline)

    return resampler.Execute(itk_image)


def normalize_image(image_np, percentile=(0.5, 99.5)):
    low, high = np.percentile(image_np, percentile)
    image_np = np.clip(image_np, low, high)
    image_np = (image_np - np.mean(image_np)) / np.std(image_np)
    return image_np

def save_nifti(array_np, reference_img, output_path):
    out_img = sitk.GetImageFromArray(array_np)
    out_img.SetSpacing(reference_img.GetSpacing())
    out_img.SetOrigin(reference_img.GetOrigin())
    out_img.SetDirection(reference_img.GetDirection())
    sitk.WriteImage(out_img, output_path)

def Padding(array_np, multiple=16):
    """
    Pad array so that each spatial dimension is a multiple of `multiple`.
    Returns padded array and padding info: [(pad_before, pad_after), ...]
    """
    shape = array_np.shape
    pad_width = []
    
    for dim in shape:
        total_pad = (multiple - dim % multiple) % multiple
        pad_before = total_pad // 2
        pad_after = total_pad - pad_before
        pad_width.append((pad_before, pad_after))

    padded = np.pad(array_np, pad_width, mode="constant", constant_values=0)
    return padded, pad_width

def process_and_save(image_path, mask_path, save_dir, target_spacing=(1.0, 1.3, 1.3), is_train=True):
    # Load image
    img_itk = sitk.ReadImage(image_path)
    mask_itk = sitk.ReadImage(mask_path)

    # Resample
    img_resampled = resample_image(img_itk, target_spacing, is_label=False)
    mask_resampled = mask_itk
    if is_train:
        mask_resampled = resample_image(mask_itk, target_spacing, is_label=True)

    # Save resampled image as numpy array

    img_np = sitk.GetArrayFromImage(img_resampled)
    scaler = MinMaxScaler()
    img_np = scaler.fit_transform(
        img_np.reshape(-1, img_np.shape[-1])
    ).reshape(img_np.shape)
    print(f"Before Padding size: {img_np.shape}")
    # Padding to multiple of 16
    img_np, pad_list = Padding(img_np, 20)
    print(f"Padding size: {img_np.shape}, padding info: {pad_list}")
    np.save(os.path.join(save_dir, "GED4.npy"), img_np)
    
    # Save resampled mask as numpy array
    mask_np = sitk.GetArrayFromImage(mask_resampled)
    scaler = MinMaxScaler()
    mask_np = scaler.fit_transform(
        mask_np.reshape(-1, mask_np.shape[-1])
    ).reshape(mask_np.shape)    
    if is_train:
        # Padding to multiple of 16
        mask_np, pad_list = Padding(mask_np, 20)
        print(f"Mask Padding size: {mask_np.shape}, padding info: {pad_list}")
    np.save(os.path.join(save_dir, "mask_GED4.npy"), mask_np)

    # Save original affine
    metadata = {
        "original_spacing": img_itk.GetSpacing(),
        "original_origin": img_itk.GetOrigin(),
        "original_direction": img_itk.GetDirection(),
        "original_size": img_itk.GetSize(),
        "padding_info": pad_list,
    }
    with open(os.path.join(save_dir, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent = 4)
    
    # Save
    # os.makedirs(save_dir, exist_ok=True)
    # save_nifti(img_np, img_resampled, os.path.join(save_dir, "GED4_new.nii.gz"))

    # print(f"Saved processed image and mask to {save_dir}")

# === Example usage ===
# image_path = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25/30labeled/1002-B1-S1/GED4.nii.gz"
# mask_path = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25/30labeled/1002-B1-S1/mask_GED4.nii.gz"
# output_dir = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25"

# process_and_save(image_path, mask_path, output_dir)


In [2]:
import os
import glob
import SimpleITK as sitk

data_dir = "/data/hdc/jincan/long_seg/MICCAI25/for_unet/train"
output_dir = "/data/hdc/jincan/long_seg/MICCAI25/Resampling"
os.makedirs(output_dir, exist_ok=True)
output_dir = os.path.join(output_dir, "train")
os.makedirs(output_dir, exist_ok=True)
dir_list = os.listdir(data_dir)

for idx in range(len(dir_list)):
    image_path = os.path.join(data_dir, dir_list[idx], "GED4.nii.gz")
    mask_path = os.path.join(data_dir, dir_list[idx], "mask_GED4.nii.gz")

    save_dir = os.path.join(output_dir, dir_list[idx])
    os.makedirs(save_dir, exist_ok=True)

    process_and_save(image_path, mask_path, save_dir, is_train=True)

data_dir = "/data/hdc/jincan/long_seg/MICCAI25/for_unet/val"
output_dir = "/data/hdc/jincan/long_seg/MICCAI25/Resampling"
output_dir = os.path.join(output_dir, "val")
os.makedirs(output_dir, exist_ok=True)
dir_list = os.listdir(data_dir)

for idx in range(len(dir_list)):
    image_path = os.path.join(data_dir, dir_list[idx], "GED4.nii.gz")
    mask_path = os.path.join(data_dir, dir_list[idx], "mask_GED4.nii.gz")
    
    save_dir = os.path.join(output_dir, dir_list[idx])
    os.makedirs(save_dir, exist_ok=True)

    process_and_save(image_path, mask_path, save_dir, is_train=False)


Before Padding size: (154, 308, 400)
Padding size: (160, 320, 400), padding info: [(3, 3), (6, 6), (0, 0)]
Mask Padding size: (160, 320, 400), padding info: [(3, 3), (6, 6), (0, 0)]
Before Padding size: (181, 308, 400)
Padding size: (200, 320, 400), padding info: [(9, 10), (6, 6), (0, 0)]
Mask Padding size: (200, 320, 400), padding info: [(9, 10), (6, 6), (0, 0)]
Before Padding size: (151, 309, 402)
Padding size: (160, 320, 420), padding info: [(4, 5), (5, 6), (9, 9)]
Mask Padding size: (160, 320, 420), padding info: [(4, 5), (5, 6), (9, 9)]
Before Padding size: (151, 309, 402)
Padding size: (160, 320, 420), padding info: [(4, 5), (5, 6), (9, 9)]
Mask Padding size: (160, 320, 420), padding info: [(4, 5), (5, 6), (9, 9)]
Before Padding size: (173, 308, 400)
Padding size: (180, 320, 400), padding info: [(3, 4), (6, 6), (0, 0)]
Mask Padding size: (180, 320, 400), padding info: [(3, 4), (6, 6), (0, 0)]
Before Padding size: (176, 292, 380)
Padding size: (180, 300, 380), padding info: [(2, 2

In [27]:
data_dir = "/data/hdc/jincan/long_seg/MICCAI25/for_unet/val"
output_dir = "/data/hdc/jincan/long_seg/MICCAI25/Resampling"
output_dir = os.path.join(output_dir, "val")
os.makedirs(output_dir, exist_ok=True)
dir_list = os.listdir(data_dir)

for idx in range(len(dir_list)):
    image_path = os.path.join(data_dir, dir_list[idx], "GED4.nii.gz")
    mask_path = os.path.join(data_dir, dir_list[idx], "mask_GED4.nii.gz")
    
    save_dir = os.path.join(output_dir, dir_list[idx])
    os.makedirs(save_dir, exist_ok=True)

    process_and_save(image_path, mask_path, save_dir, is_train=False)

Before Padding size: (172, 238, 380)
Padding size: (180, 240, 380), padding info: [(4, 4), (1, 1), (0, 0)]
Before Padding size: (166, 238, 380)
Padding size: (180, 240, 380), padding info: [(7, 7), (1, 1), (0, 0)]
Before Padding size: (148, 238, 380)
Padding size: (160, 240, 380), padding info: [(6, 6), (1, 1), (0, 0)]
Before Padding size: (185, 238, 380)
Padding size: (200, 240, 380), padding info: [(7, 8), (1, 1), (0, 0)]
Before Padding size: (166, 238, 380)
Padding size: (180, 240, 380), padding info: [(7, 7), (1, 1), (0, 0)]
Before Padding size: (136, 154, 297)
Padding size: (140, 160, 300), padding info: [(2, 2), (3, 3), (1, 2)]
Before Padding size: (172, 238, 380)
Padding size: (180, 240, 380), padding info: [(4, 4), (1, 1), (0, 0)]
Before Padding size: (146, 292, 380)
Padding size: (160, 300, 380), padding info: [(7, 7), (4, 4), (0, 0)]
Before Padding size: (172, 238, 380)
Padding size: (180, 240, 380), padding info: [(4, 4), (1, 1), (0, 0)]
Before Padding size: (151, 292, 380)


In [22]:
import numpy as np

Test_Path = "/data/hdc/jincan/long_seg/MICCAI25/Resampling/train"
dir_list = os.listdir(Test_Path)
A_list = []
B1_list = []
B2_list = []
for idx in range(len(dir_list)):
    image_path = os.path.join(Test_Path, dir_list[idx], "GED4.npy")
    image_np = np.load(image_path)
    mask_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4.npy")
    mask_np = np.load(mask_path)
    if "A" in dir_list[idx]:
        A_list.append(mask_np.shape)
    elif "B1" in dir_list[idx]:
        B1_list.append(mask_np.shape)
    elif "B2" in dir_list[idx]:
        B2_list.append(mask_np.shape)
    # print(f"Mask shape: {mask_np.shape}, unique values: {np.unique(mask_np)}")

Test_Path = "/data/hdc/jincan/long_seg/MICCAI25/Resampling/val"
dir_list = os.listdir(Test_Path)
for idx in range(len(dir_list)):
    image_path = os.path.join(Test_Path, dir_list[idx], "GED4.npy")
    image_np = np.load(image_path)
    mask_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4.npy")
    mask_np = np.load(mask_path)
    if "A" in dir_list[idx]:
        A_list.append(mask_np.shape)
    elif "B1" in dir_list[idx]:
        B1_list.append(mask_np.shape)
    elif "B2" in dir_list[idx]:
        B2_list.append(mask_np.shape)
    # print(f"Mask shape: {mask_np.shape}, unique values: {np.unique(mask_np)}")

print(f"A list shape: {A_list}")
print(f"B1 list shape: {B1_list}")
print(f"B2 list shape: {B2_list}")

# mask_original = sitk.ReadImage(mask_original_path)
# mask_resampled = sitk.ReadImage(mask_resampled_path)
# mask_resampled_back = sitk.ReadImage(mask_resampled_back_path)

# # Convert to numpy arrays for visualization or further processing
# mask_original_np = sitk.GetArrayFromImage(mask_original)
# mask_resampled_np = sitk.GetArrayFromImage(mask_resampled)
# mask_resampled_back_np = sitk.GetArrayFromImage(mask_resampled_back)

# # Calculate the DICE bewtween the original and resampled back masks
# def dice_coefficient(mask1, mask2):
#     intersection = np.sum(mask1 * mask2)
#     return 2.0 * intersection / (np.sum(mask1) + np.sum(mask2))
# dice_original_resampled_back = dice_coefficient(mask_original_np, mask_resampled_back_np)
# print(f"DICE coefficient between original and resampled back mask: {dice_original_resampled_back:.4f}")


A list shape: [(64, 520, 640), (72, 260, 320), (64, 260, 320), (80, 260, 320), (72, 260, 320), (59, 169, 250), (64, 520, 640), (38, 320, 320), (64, 520, 640), (28, 320, 320)]
B1 list shape: [(160, 320, 420), (160, 320, 420), (180, 300, 380), (160, 320, 420), (180, 300, 380), (180, 300, 380), (180, 300, 380), (180, 300, 380), (160, 320, 420), (160, 320, 420)]
B2 list shape: [(160, 320, 400), (200, 320, 400), (180, 320, 400), (160, 320, 400), (180, 320, 400), (180, 320, 400), (180, 320, 400), (180, 300, 400), (180, 320, 400), (180, 320, 400)]


In [1]:
import os
base_data = "/data/hdc/jincan/long_seg/MICCAI25/"
list = os.listdir(base_data)
print(list)

['mask.nii.gz', 'Resampling', '30labeled', 'for_nnunet', 'for_unet', 'image.nii.gz', 'liver_tot', 'unlabeled']
